In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mediapipe
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
grpcio

In [ ]:
!pip install tensorflow==2.18.0 numpy==1.26.4 mediapipe==0.10.21 protobuf==4.25.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.7
    Uninstalling protobuf-4.25.7:
      Successfully uninstalled protobuf-4.25.7
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's

In [ ]:
!pip uninstall mediapipe -y
!pip install mediapipe==0.10.7


Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


# **SimpleCNN**





In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm
import os

# Khởi tạo mô-đun MediaPipe
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

def extract_landmarks_with_overlay(image_path, size=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        return None

    # Phát hiện khuôn mặt bằng BlazeFace
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detection_results = face_detection.process(image_rgb)

    if not detection_results.detections:
        return None

    # Trích xuất 468 điểm bằng Face Mesh (dùng ảnh gốc, không crop)
    mesh_results = face_mesh.process(image_rgb)
    if mesh_results.multi_face_landmarks:
        for landmark in mesh_results.multi_face_landmarks:
            for point in landmark.landmark:
                x = int(point.x * image.shape[1])
                y = int(point.y * image.shape[0])
                cv2.circle(image, (x, y), 1, (255, 0, 255), -1)
    else:
        return None

    image = cv2.resize(image, size)
    return image

In [ ]:
def create_dataset_with_landmarks(source_dir, target_dir, size=(224, 224)):
    os.makedirs(target_dir, exist_ok=True)

    for class_name in ['real', 'fake']:
        src_folder = os.path.join(source_dir, class_name)
        tgt_folder = os.path.join(target_dir, class_name)
        os.makedirs(tgt_folder, exist_ok=True)

        for filename in tqdm(os.listdir(src_folder)):
            tgt_path = os.path.join(tgt_folder, filename)
            if os.path.exists(tgt_path):
                continue

            src_path = os.path.join(src_folder, filename)
            output = extract_landmarks_with_overlay(src_path, size=size)
            if output is not None:
                cv2.imwrite(tgt_path, output)

# Gọi tạo dataset
create_dataset_with_landmarks('/content/real_vs_fake/real-vs-fake/train', '/content/data_landmark_overlay/train')
create_dataset_with_landmarks('/content/real_vs_fake/real-vs-fake/valid', '/content/data_landmark_overlay/valid')
create_dataset_with_landmarks('/content/real_vs_fake/real-vs-fake/test', '/content/data_landmark_overlay/test')

100%|██████████| 10000/10000 [03:43<00:00, 44.72it/s]


In [ ]:
# Load dữ liệu

from tensorflow.keras.preprocessing.image import ImageDataGenerator

size = (224, 224)
datagen = ImageDataGenerator(rescale=1./255)
train = datagen.flow_from_directory('/content/data_landmark_overlay/train',
                                    target_size=size,
                                    class_mode='binary',
                                    batch_size=32,
                                    shuffle=True)

val = datagen.flow_from_directory('/content/data_landmark_overlay/valid',
                                  target_size=size,
                                  class_mode='binary',
                                  batch_size=32,
                                  shuffle=False)

test = datagen.flow_from_directory('/content/data_landmark_overlay/test',
                                   target_size=size,
                                   class_mode='binary',
                                   batch_size=32,
                                   shuffle=False)






Found 97623 images belonging to 2 classes.
Found 19525 images belonging to 2 classes.
Found 19516 images belonging to 2 classes.


In [ ]:
# 224 x 224 train model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    Dropout(0.2),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    Dropout(0.3),
    MaxPooling2D(2, 2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    Dropout(0.3),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])  

from keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='model_weights.weights.h5', save_weights_only=True,
                             monitor='val_loss', mode='min', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)

history = model.fit(train, validation_data=val,
                    epochs=30,
                    callbacks=[checkpoint, early_stopping, reduce_lr])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 298s 94ms/step - accuracy: 0.7031 - loss: 0.5613 - val_accuracy: 0.8378 - val_loss: 0.4391 - learning_rate: 1.0000e-04
Epoch 2/30
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 237s 78ms/step - accuracy: 0.8534 - loss: 0.3393 - val_accuracy: 0.8565 - val_loss: 0.3613 - learning_rate: 1.0000e-04
Epoch 3/30
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 258s 76ms/step - accuracy: 0.9002 - loss: 0.2448 - val_accuracy: 0.9120 - val_loss: 0.2824 - learning_rate: 1.0000e-04
Epoch 4/30
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 240s 79ms/step - accuracy: 0.9297 - loss: 0.1792 - val_accuracy: 0.9347 - val_loss: 0.2290 - learning_rate: 1.0000e-04
Epoch 5/30
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 240s 79ms/step - accuracy: 0.9489 - loss: 0.1338 - val_accuracy: 0.9418 - val_loss: 0.1744 - learning_rate: 1.0000e-04
Epoch 6/30
3051/3051 ━━━━━━━━━━━━━━━━━━━━ 254s 83ms/step - accuracy: 0.9594 - loss: 0.1044 - val_accuracy: 0.9436 - val_loss: 0.1639 - learning_rate: 1.0000e-04
Epoch 7/30
3051/3051 ━━━━━━━━━━━━━

In [ ]:
model.save("/content/drive/MyDrive/deepfake_coban_model.keras")
model.save_weights("/content/drive/MyDrive/coban.weights.h5")

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,351,621 (295.07 MB)

 Trainable params: 25,783,873 (98.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 51,567,748 (196.72 MB)